In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras.ops as K

In [ ]:
def cbam_block(inputs, ratio=8):
    channel = inputs.shape[-1]

   
    avg_pool = layers.GlobalAveragePooling2D(keepdims=True)(inputs)
    max_pool = layers.GlobalMaxPooling2D(keepdims=True)(inputs)

    mlp = keras.Sequential([
        layers.Dense(channel // ratio, activation='relu'),
        layers.Dense(channel)
    ])

    avg_out = mlp(avg_pool)
    max_out = mlp(max_pool)

    channel_attention = layers.Activation("sigmoid")(avg_out + max_out)
    channel_refined = layers.Multiply()([inputs, channel_attention])

    
    avg_spatial = K.mean(channel_refined, axis=-1, keepdims=True)
    max_spatial = K.max(channel_refined, axis=-1, keepdims=True)

    concat = layers.Concatenate(axis=-1)([avg_spatial, max_spatial])

    spatial_attention = layers.Conv2D(
        filters=1,
        kernel_size=7,
        padding="same",
        activation="sigmoid"
    )(concat)

    refined = layers.Multiply()([channel_refined, spatial_attention])
    return refined

In [3]:
def build_model():
    inputs = keras.Input(shape=(224, 224, 3))

    base = keras.applications.EfficientNetB0(
        include_top=False,
        input_shape=(224, 224, 3),
        weights="imagenet"
    )

    x = keras.applications.efficientnet.preprocess_input(inputs)
    x = base(x)

    x = cbam_block(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(7, activation="softmax")(x)

    model = keras.Model(inputs, outputs)
    return model

model = build_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb0      │ (None, 7, 7,      │  4,049,571 │ input_layer[0][0] │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1, 1,      │          0 │ efficientnetb0[0… │
│ (GlobalAveragePool… │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 1, 1,      │          0 │ efficientnetb0[0… │
│ (GlobalMaxPooling2… │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 1, 1,      │    411,040 │ global_average_p… │
│ (Sequential)        │ 1280)             │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 1, 1,      │          0 │ sequential[0][0], │
│                     │ 1280)             │            │ sequential[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 1, 1,      │          0 │ add[0][0]         │
│ (Activation)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 7, 7,      │          0 │ efficientnetb0[0… │
│                     │ 1280)             │            │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mean (Mean)         │ (None, 7, 7, 1)   │          0 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max (Max)           │ (None, 7, 7, 1)   │          0 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 7, 7, 2)   │          0 │ mean[0][0],       │
│ (Concatenate)       │                   │            │ max[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 7, 7, 1)   │         99 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 7, 7,      │          0 │ multiply[0][0],   │
│ (Multiply)          │ 1280)             │            │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ multiply_1[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1280)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 7)         │      8,967 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,469,677 (17.05 MB)

 Trainable params: 4,427,654 (16.89 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [ ]:
data_dir = "C://Users//manis//Downloads//FER-13//train" 

batch_size = 32
img_size = (224, 224)

dataset = keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=True,
    validation_split=0.2,    
    subset="both",           
    seed=42
)

train_ds, val_ds = dataset

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                        num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Using 5741 files for validation.


In [5]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_cbam_model.h5",
        monitor="val_accuracy",
        save_best_only=True
    ),
    keras.callbacks.EarlyStopping(
        patience=8,
        monitor="val_loss",
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        factor=0.3,
        monitor="val_loss",
        patience=3,
        min_lr=1e-6
    )
]

In [6]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks
)

Epoch 1/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3947 - loss: 1.5679

718/718 ━━━━━━━━━━━━━━━━━━━━ 2373s 3s/step - accuracy: 0.4735 - loss: 1.3757 - val_accuracy: 0.5712 - val_loss: 1.1149 - learning_rate: 1.0000e-04
Epoch 2/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5776 - loss: 1.1256

718/718 ━━━━━━━━━━━━━━━━━━━━ 2006s 3s/step - accuracy: 0.5920 - loss: 1.0934 - val_accuracy: 0.6239 - val_loss: 1.0093 - learning_rate: 1.0000e-04
Epoch 3/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6211 - loss: 1.0145

718/718 ━━━━━━━━━━━━━━━━━━━━ 2118s 3s/step - accuracy: 0.6290 - loss: 0.9925 - val_accuracy: 0.6426 - val_loss: 0.9727 - learning_rate: 1.0000e-04
Epoch 4/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6506 - loss: 0.9312

718/718 ━━━━━━━━━━━━━━━━━━━━ 2123s 3s/step - accuracy: 0.6543 - loss: 0.9200 - val_accuracy: 0.6509 - val_loss: 0.9507 - learning_rate: 1.0000e-04
Epoch 5/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6813 - loss: 0.8668

...

718/718 ━━━━━━━━━━━━━━━━━━━━ 2132s 3s/step - accuracy: 0.7042 - loss: 0.7988 - val_accuracy: 0.6589 - val_loss: 0.9502 - learning_rate: 1.0000e-04
Epoch 7/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7187 - loss: 0.7589

718/718 ━━━━━━━━━━━━━━━━━━━━ 2125s 3s/step - accuracy: 0.7237 - loss: 0.7410 - val_accuracy: 0.6643 - val_loss: 0.9497 - learning_rate: 1.0000e-04
Epoch 8/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7417 - loss: 0.7091

718/718 ━━━━━━━━━━━━━━━━━━━━ 2016s 3s/step - accuracy: 0.7449 - loss: 0.6980 - val_accuracy: 0.6734 - val_loss: 0.9545 - learning_rate: 1.0000e-04
Epoch 9/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 2093s 3s/step - accuracy: 0.7812 - loss: 0.6044 - val_accuracy: 0.6725 - val_loss: 0.9774 - learning_rate: 3.0000e-05
Epoch 10/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 6438s 9s/step - accuracy: 0.7899 - loss: 0.5749 - val_accuracy: 0.6710 - val_loss: 0.9950 - learning_rate: 3.0000e-05
Epoch 11/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 8418s 12s/step - accuracy: 0.7974 - loss: 0.5545 - val_accuracy: 0.6715 - val_loss: 1.0106 - learning_rate: 3.0000e-05
Epoch 12/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8022 - loss: 0.5480

718/718 ━━━━━━━━━━━━━━━━━━━━ 1950s 3s/step - accuracy: 0.8101 - loss: 0.5267 - val_accuracy: 0.6750 - val_loss: 1.0062 - learning_rate: 9.0000e-06
Epoch 13/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8075 - loss: 0.5305

718/718 ━━━━━━━━━━━━━━━━━━━━ 1948s 3s/step - accuracy: 0.8158 - loss: 0.5092 - val_accuracy: 0.6751 - val_loss: 1.0111 - learning_rate: 9.0000e-06


In [7]:
model.save("cbam_efficientnetb0_v1.h5")

In [ ]:
model = build_model() 
model.load_weights("cbam_efficientnetb0_v1.h5")


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    initial_epoch=13,  
    epochs=30,          
    callbacks=callbacks
)


Epoch 14/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 2101s 3s/step - accuracy: 0.7055 - loss: 0.7939 - val_accuracy: 0.6569 - val_loss: 0.9528 - learning_rate: 1.0000e-04
Epoch 15/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 1978s 3s/step - accuracy: 0.7239 - loss: 0.7459 - val_accuracy: 0.6666 - val_loss: 0.9628 - learning_rate: 1.0000e-04
Epoch 16/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 1978s 3s/step - accuracy: 0.7399 - loss: 0.7000 - val_accuracy: 0.6682 - val_loss: 0.9788 - learning_rate: 1.0000e-04
Epoch 17/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7643 - loss: 0.6363

718/718 ━━━━━━━━━━━━━━━━━━━━ 2032s 3s/step - accuracy: 0.7746 - loss: 0.6102 - val_accuracy: 0.6764 - val_loss: 0.9654 - learning_rate: 3.0000e-05
Epoch 18/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 1988s 3s/step - accuracy: 0.7898 - loss: 0.5798 - val_accuracy: 0.6760 - val_loss: 0.9820 - learning_rate: 3.0000e-05
Epoch 19/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7894 - loss: 0.5698

718/718 ━━━━━━━━━━━━━━━━━━━━ 2062s 3s/step - accuracy: 0.7934 - loss: 0.5604 - val_accuracy: 0.6776 - val_loss: 0.9916 - learning_rate: 3.0000e-05
Epoch 20/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7951 - loss: 0.5485

718/718 ━━━━━━━━━━━━━━━━━━━━ 2033s 3s/step - accuracy: 0.8052 - loss: 0.5277 - val_accuracy: 0.6797 - val_loss: 0.9919 - learning_rate: 9.0000e-06
Epoch 21/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8006 - loss: 0.5403

718/718 ━━━━━━━━━━━━━━━━━━━━ 2023s 3s/step - accuracy: 0.8109 - loss: 0.5164 - val_accuracy: 0.6800 - val_loss: 0.9962 - learning_rate: 9.0000e-06
